In [2]:
!curl localhost:9200

{
  "name" : "25a7202b13f6",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "PrEqG-e7Sx2XJDDfGMjPdw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [11]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
from tqdm.auto import tqdm

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:24<00:00, 38.06it/s]


In [24]:
def elastic_search_response(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    return es_client.search(index=index_name, body=search_query)

def elastic_search(query):
    response = elastic_search_response(query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [29]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            }
        }
    }
}
result_q3 = es_client.search(index=index_name, body=search_query)

result_q3['hits']['hits'][0]['_score']

84.050095

In [30]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

result_q4 = es_client.search(index=index_name, body=search_query)

result_q4['hits']['hits'][2]['_source']["question"]

'How do I copy files from a different folder into docker container’s working directory?'

In [36]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc_json in result_q4['hits']['hits']:
    doc = doc_json['_source']
    context += context_template.format(question=doc["question"], text=doc["text"]).strip() + "\n\n"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

final_prompt = prompt_template.format(question=query, context=context).strip()

len(final_prompt)

1462

In [38]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(final_prompt))

322

In [48]:
from openai import OpenAI

client = OpenAI()

In [49]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": final_prompt}]
)

In [57]:
print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. First, you need to find the `container-id` of the running container by using:

```sh
docker ps
```

Once you have the `container-id`, you can execute a command in the container by using:

```sh
docker exec -it <container-id> <command>
```

For example, to start a bash session in the container, you would use:

```sh
docker exec -it <container-id> bash
```


In [59]:
input_tokens = 150
output_tokens = 250
requests = 1000
input_cost = 0.005
output_cost = 0.015

price = (input_tokens * input_cost + output_tokens * output_cost) * requests / 1000
print(price)

4.5


In [61]:
input_tokens = len(encoding.encode(final_prompt))
output_tokens = len(encoding.encode(response.choices[0].message.content))

price = (input_tokens * input_cost + output_tokens * output_cost) * requests / 1000
print(price)

3.2750000000000004
